In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
# seaborn is going to be your friend
import seaborn as sns

In [2]:
df = pd.read_csv("netgen.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'netgen.csv'

In [ ]:
budget_df = pd.read_csv("zippedData/tn.movie_budgets.csv.gz")

In [ ]:
budget_df.info()

In [ ]:
budget_df['production_budget'] = budget_df['production_budget'].str.replace(",","")
budget_df['production_budget'] = budget_df['production_budget'].str.replace("$","")
budget_df['domestic_gross'] = budget_df['domestic_gross'].str.replace(",","")
budget_df['domestic_gross'] = budget_df['domestic_gross'].str.replace("$","")
budget_df['worldwide_gross'] = budget_df['worldwide_gross'].str.replace(",","")
budget_df['worldwide_gross'] = budget_df['worldwide_gross'].str.replace("$","")

In [ ]:
budget_df['domestic_gross'] = budget_df['domestic_gross'].apply(lambda x: int(x))
budget_df['production_budget'] = budget_df['production_budget'].apply(lambda x: int(x))
budget_df['worldwide_gross'] = budget_df['worldwide_gross'].apply(lambda x: int(x))

In [ ]:
budget_df.info()

In [ ]:
con = sqlite3.connect("zippedData/im.db/im.db")

cur = con.cursor()

In [ ]:
%%bash 
sqlite3 zippedData/im.db/im.db
.schema

In [ ]:
persons_df = pd.read_sql("""
SELECT *
FROM persons
""",con)

In [ ]:
persons_df

In [ ]:
pri_per_df = pd.read_sql("""
SELECT *
FROM principals
LEFT JOIN persons
ON principals.person_id = persons.person_id
""",con)

In [ ]:
pri_per_df

In [ ]:
pri_per_df

In [ ]:
reviews_df = pd.read_csv("review_dataframe.csv")

In [ ]:
reviews_df

In [ ]:
reviews_df = reviews_df.set_index('movie_id')
pri_per_df = pri_per_df.set_index('movie_id')

In [ ]:
reviews_df

In [ ]:
pri_per_df

In [ ]:
imdb_df = pri_per_df.merge(reviews_df, on = "movie_id", how = "left")

In [ ]:
imdb_df.info()

In [ ]:
imdb_df = imdb_df.drop(["job",'characters','birth_year','death_year','Unnamed: 0'],axis = 1)

In [ ]:
imdb_df = imdb_df.dropna(subset = ['genres'])

In [ ]:
imdb_df["movie_id"] = imdb_df.index

In [ ]:
imdb_df = imdb_df.set_index("primary_title", drop = False)

In [ ]:
cols = imdb_df.columns.tolist()

In [ ]:
imdb_df

In [ ]:
cols

In [ ]:
cols_alt = ['movie_id', 'primary_title', 'original_title', 'genres', 'start_year', 'runtime_minutes', 'averagerating', 'ratingbyvotes', 'numvotes', 'ordering', 'person_id', 'primary_name', 'person_id', 'category', 'primary_profession']

In [ ]:
cols_alt = ['movie_id', 'primary_title', 'original_title', 'genres', 'start_year', 'runtime_minutes', 'averagerating', 'numvotes', 'primary_name', 'category']

In [ ]:
imdb_df_alt = imdb_df[cols_alt]

Analyzing data and adding budget below

In [ ]:
imdb_df.to_csv('imdb_masterdata.csv')

In [ ]:
imdb_df_alt.to_csv('imdb_masterdata.csv')

In [ ]:
sns.pairplot(imdb_df_alt, corner = True)
plt.show()

In [ ]:
imdb_df_alt

In [ ]:
sns.histplot(x = 'averagerating', hue = 'genres', element="step", data = imdb_df_alt, binwidth = 1)
plt.show()

In [ ]:
sns.violinplot(x = 'genres', y = 'averagerating',
                
               data = imdb_df_alt)
plt.show()

In [ ]:
budget_df = budget_df.rename(columns={"movie_id": "primary_title"})

In [ ]:
budget_df.info()

In [ ]:
imdb_df_alt.reset_index(drop = True, inplace = True)

In [ ]:
imdb_df_alt.info()

In [ ]:
imdb_budget_df = imdb_df_alt.merge(budget_df, on = 'primary_title', how = "left")

In [ ]:
imdb_budget_df = imdb_budget_df.dropna(subset = ['production_budget'])

In [ ]:
imdb_budget_df["revenue"] = imdb_budget_df["worldwide_gross"] + imdb_budget_df["production_budget"]

In [ ]:
imdb_budget_df['profit_margin'] = imdb_budget_df["worldwide_gross"] / imdb_budget_df["revenue"]

In [ ]:
imdb_budget_df.to_csv('imdb_budget_data.csv')

In [ ]:
imdb_budget_df['release_date'] = imdb_budget_df['release_date'].astype('datetime64')

In [ ]:
imdb_budget_df

In [ ]:
sns.pairplot(imdb_budget_df, corner = True, hue = 'genres')
plt.show()